In [1]:
from bokeh.io import output_notebook, show
from bokeh.palettes import Colorblind, RdGy
from bokeh.plotting import figure, show, output_file
from collections import OrderedDict
from itertools import cycle
from math import log, sqrt

import numpy as np
import pandas as pd
from six.moves import cStringIO as StringIO

In [2]:
output_notebook()

Loading BokehJS ...

# Data Load

In [3]:
excelFile = pd.ExcelFile("Ag_Census_Map_data_07172015.xlsx")

agri_df = {}
for sheet_name in excelFile.sheet_names:
    agri_df[sheet_name] = excelFile.parse(sheet_name)

var_lookup = agri_df['Variable Lookup']

var_lookup[var_lookup.Category == 'Economics'].head()

,MapID,Category,Subcategory,MAPTITLE
81,y12_M097,Economics,"Buildings, Machinery and Equipment on Operation","Estimated Market Value of Land and Buildings, ..."
82,y12_M098,Economics,"Buildings, Machinery and Equipment on Operation",Estimated Market Value of All Machinery and Eq...
83,y12_M267,Economics,"Buildings, Machinery and Equipment on Operation",Average Number of Harvested Cropland Acres per...
84,y12_M268,Economics,"Buildings, Machinery and Equipment on Operation","Average Number of Corn, Soybean, and Wheat Har..."
85,y12_M050,Economics,Farm Production Expenses,Average Total Farm Production Expenses per Far...


In [4]:
agri_df.keys()

dict_keys(['Readme', 'County Names', 'Variable Lookup', 'Crops and Plants', 'Economics', 'Farms', 'Livestock and Animals', 'Operators'])

In [5]:
agri_df['County Names'].head()

,FIPSTEXT,CountyName,Entity,StateName
0,1001,Autauga,County,Alabama
1,1003,Baldwin,County,Alabama
2,1005,Barbour,County,Alabama
3,1007,Bibb,County,Alabama
4,1009,Blount,County,Alabama


In [6]:
agri_df['Economics'].head()

,FIPS,FIPSTEXT,y12_M009_valueText,y12_M009_classRange,y12_M009_valueNumeric,y12_M010_valueText,y12_M010_classRange,y12_M010_valueNumeric,y12_M011_valueText,y12_M011_classRange,...,y12_M258_valueNumeric,y12_M267_valueText,y12_M267_classRange,y12_M267_valueNumeric,y12_M268_valueText,y12_M268_classRange,y12_M268_valueNumeric,y12_M373_valueText,y12_M373_classRange,y12_M373_valueNumeric
0,0,0,56.61,NaN,56.61,31.58,NaN,31.58,11.82,NaN,...,9925.0,75.38,NaN,75.38,613.21,NaN,613.21,217835.92,NaN,217835.92
1,1001,1001,66.32,60 - 74,66.32,29.56,20 - 29,29.56,4.11,1 - 4,...,10039.0,40.65,25 - 49,40.65,182.57,50 - 249,182.57,0.00,"Less than 50,000",0.00
2,1003,1003,64.81,60 - 74,64.81,26.29,20 - 29,26.29,8.90,5 - 9,...,10036.0,48.79,25 - 49,48.79,370.05,250 - 499,370.05,4275.50,"Less than 50,000",4275.50
3,1005,1005,67.25,60 - 74,67.25,24.52,20 - 29,24.52,8.23,5 - 9,...,5059.0,24.51,Less than 25,24.51,(D),50 - 249,NaN,(D),"Less than 50,000",NaN
4,1007,1007,76.19,75 or more,76.19,22.75,20 - 29,22.75,1.06,1 - 4,...,5103.0,26.51,25 - 49,26.51,(D),750 or more,NaN,0.00,"Less than 50,000",0.00


# Merge Locations with Economics

In [7]:
economics = pd.merge(agri_df['Economics'], agri_df['County Names'],
                 how='left', on='FIPSTEXT').drop(['FIPS', 'FIPSTEXT'], axis=1)

In [8]:
economics.head()

,y12_M009_valueText,y12_M009_classRange,y12_M009_valueNumeric,y12_M010_valueText,y12_M010_classRange,y12_M010_valueNumeric,y12_M011_valueText,y12_M011_classRange,y12_M011_valueNumeric,y12_M013_valueText,...,y12_M267_valueNumeric,y12_M268_valueText,y12_M268_classRange,y12_M268_valueNumeric,y12_M373_valueText,y12_M373_classRange,y12_M373_valueNumeric,CountyName,Entity,StateName
0,56.61,NaN,56.61,31.58,NaN,31.58,11.82,NaN,11.82,187097,...,75.38,613.21,NaN,613.21,217835.92,NaN,217835.92,NaN,NaN,NaN
1,66.32,60 - 74,66.32,29.56,20 - 29,29.56,4.11,1 - 4,4.11,50929,...,40.65,182.57,50 - 249,182.57,0.00,"Less than 50,000",0.00,Autauga,County,Alabama
2,64.81,60 - 74,64.81,26.29,20 - 29,26.29,8.90,5 - 9,8.90,137069,...,48.79,370.05,250 - 499,370.05,4275.50,"Less than 50,000",4275.50,Baldwin,County,Alabama
3,67.25,60 - 74,67.25,24.52,20 - 29,24.52,8.23,5 - 9,8.23,182761,...,24.51,(D),50 - 249,NaN,(D),"Less than 50,000",NaN,Barbour,County,Alabama
4,76.19,75 or more,76.19,22.75,20 - 29,22.75,1.06,1 - 4,1.06,(D),...,26.51,(D),750 or more,NaN,0.00,"Less than 50,000",0.00,Bibb,County,Alabama


# Create Regions

In [9]:
new_england = economics['StateName'].isin(
    ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont'])
mid_atlantic = economics['StateName'].isin(
    ['New Jersey', 'New York', 'Pennsylvania'])
north_east_central = economics['StateName'].isin(
    ['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin'])
north_west_central = economics['StateName'].isin(
    ['Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota'])
south_atlantic = economics['StateName'].isin(
    ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia',
     'District of Columbia', 'West Virginia'])
south_east_central = economics['StateName'].isin(
    ['Alabama', 'Kentucky', 'Mississippi', 'Tennessee'])
south_west_central = economics['StateName'].isin(
    ['Arkansas', 'Louisiana', 'Oklahoma', 'Texas'])
mountain = economics['StateName'].isin(
    ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming'])
pacific = economics['StateName'].isin(
    ['Alaska', 'California', 'Hawaii', 'Oregon', 'Washington'])

regions = [new_england, mid_atlantic, north_east_central, north_west_central,
           south_atlantic, south_east_central, south_west_central, mountain, pacific]
region_names = ['New England', 'Mid Atlantic', 'East North-Central', 'West North-Central',
                'South Atlantic', 'East South-Central', 'West South-Central', 'Mountain', 'Pacific']

In [10]:
for region, name in zip(regions, region_names):
    economics.loc[region, 'Region'] = name

## Aggregate for % of Farms with Sales in different Categories

In [11]:
regional_economics = economics.groupby('Region').mean().reset_index()
regional_economics[['Region', 'y12_M009_valueNumeric',
                'y12_M010_valueNumeric', 'y12_M011_valueNumeric']]

,Region,y12_M009_valueNumeric,y12_M010_valueNumeric,y12_M011_valueNumeric
0,East North-Central,52.863570,33.219542,13.916842
1,East South-Central,67.433901,26.089066,6.476923
2,Mid Atlantic,53.985333,36.408733,8.939867
3,Mountain,56.514270,31.248292,11.881886
4,New England,65.333433,29.221194,5.445672
5,Pacific,56.430915,31.309366,12.258873
6,South Atlantic,65.536951,24.623975,9.839056
7,West North-Central,40.251994,37.252091,22.496110
8,West South-Central,61.877143,29.910985,8.212020


# Plot

In [12]:
# Colors
outer_colors = cycle(RdGy[3][:2])
inner_colors = iter(Colorblind[3])
region_color = OrderedDict([
    (name, next(outer_colors)) for name in regional_economics.Region
])

# Plot Size
width = 800
height = 800
inner_radius = 100
outer_radius = 310 - 10

# Radius Limits
minr = 75
maxr = 0
a = (outer_radius - inner_radius) / (minr - maxr)
b = inner_radius - a * maxr

# Radius calculation based on limits
def rad(mic):
    return a * mic + b

# Region Radii computation
big_angle = 2.0 * np.pi / (len(regional_economics) + 1)
small_angle = big_angle / 7

# Base Plot
p = figure(plot_width=width, plot_height=height, title="Average Percent of Farms w/ Total Sales in 2012",
           x_axis_type=None, y_axis_type=None,
           x_range=(-420, 420), y_range=(-420, 420),
           min_border=0, outline_line_color="black",
           background_fill_color="white")

# Remove all grids
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Annular wedges for regions
angles = np.pi/2 - big_angle/2 - regional_economics.index.to_series()*big_angle
colors = [region_color[region] for region in regional_economics.Region]
p.annular_wedge(
    0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
)

# Small wedges for values
p.annular_wedge(0, 0, inner_radius, rad(regional_economics.y12_M009_valueNumeric),
                -big_angle+angles+5*small_angle, -big_angle+angles+6*small_angle,
                color=next(inner_colors))
p.annular_wedge(0, 0, inner_radius, rad(regional_economics.y12_M010_valueNumeric),
                -big_angle+angles+3*small_angle, -big_angle+angles+4*small_angle,
                color=next(inner_colors))
p.annular_wedge(0, 0, inner_radius, rad(regional_economics.y12_M011_valueNumeric),
                -big_angle+angles+1*small_angle, -big_angle+angles+2*small_angle,
                color=next(inner_colors))

# Circular axes and lables
labels = np.array(range(75, -1, -15))
radii = a * labels + b
p.circle(0, 0, radius=radii, fill_color=None, line_color="black")
p.text(0, radii[:-1]-10, [str(r)+'%' for r in labels[:-1]],
       text_font_size="8pt", text_align="center", text_baseline="middle")

# Region labels
xr = (radii[0]+75)*np.cos(np.array(-big_angle/2 + angles))
yr = (radii[0]+75)*np.sin(np.array(-big_angle/2 + angles))
label_angle = np.array(-big_angle/2+angles)
# easier to read labels on the left side
label_angle[label_angle < -np.pi/2] += np.pi
p.text(xr, yr, regional_economics.Region, angle=label_angle,
       text_font_size="9pt", text_align="center", text_baseline="middle")

# Center Legend
p.rect([-65, -65, -65], [18, 0, -18], width=30, height=13,
       color=list(Colorblind[3]))
p.text([-40, -40, -40], [18, 0, -18], text=['% < $10,000', '% $10,000 to $249,999', '% > $250,000'],
       text_font_size="9pt", text_align="left", text_baseline="middle")

GlyphRenderer(id='1069', ...)

In [13]:
show(p)